In [4]:
%pip install rdkit

  Using cached pillow-10.4.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
   ---------------------------------------- 0.0/21.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/21.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/21.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/21.7 MB 1.6 MB/s eta 0:00:14
    --------------------------------------- 0.5/21.7 MB 4.4 MB/s eta 0:00:05
   - -------------------------------------- 1.0/21.7 MB 6.6 MB/s eta 0:00:04
   - -------------------------------------- 1.0/21.7 MB 6.6 MB/s eta 0:00:04
   - -------------------------------------- 1.0/21.7 MB 6.6 MB/s eta 0:00:04
   - -------------------------------------- 1.0/21.7 MB 6.6 MB/s eta 0:00:04
   - -------------------------------------- 1.0/21.7 MB 6.6 MB/s eta 0:00:04
   - -------------------------------------- 1.0/21.7 MB 6.6 MB/s eta 0:00:04
   -- ------------------------------------- 1.2/21.7 MB 2.7 MB/s eta 0:00:08
   --- -----------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.DataStructs import TanimotoSimilarity

In [8]:
patient = pd.read_csv("patient_SMILES.csv")
patient.head()

,patientID,gender,age,weight,prescribed drug (drugA),chem structure (drugA),newly prescribed drug (drugB),chem structure (drugB),DDI,clinical insights (recommendation),SMILES (drugA),SMILES (drugB)
0,1940200,1,64,65,POTASSIUM CATION,K,LOTEPREDNOL ETABONATE,C24H31ClO7,moderate,NaN,[K+],[H][C@@]12CC[C@](OC(=O)OCC)(C(=O)OCCl)[C@@]1(C...
1,1947086,0,55,62,ZINC SULFATE,O4SZn,POTASSIUM HYDROXIDE,HKO,no interaction found,NaN,[Zn++].[O-]S([O-])(=O)=O,[OH-].[K+]
2,1947690,0,56,55,LINAGLIPTIN,C25H28N8O2,MEBENDAZOLE,C16H13N3O3,no interaction found,NaN,CC#CCN1C(=NC2=C1C(=O)N(CC1=NC3=C(C=CC=C3)C(C)=...,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=CC=C1
3,2022991,1,54,68,POVIDONE,NaN,ETAFEDRINE,C12H19NO,no interaction found,NaN,NaN,CCN(C)[C@@H](C)[C@H](O)C1=CC=CC=C1
4,1969141,1,40,85,PRILOCAINE,C13H20N2O,SODIUM AUROTHIOMALATE,C4H3AuNa2O4S,no interaction found,NaN,CCCNC(C)C(=O)NC1=CC=CC=C1C,[Na+].[Na+].[O-]C(=O)CC(S[Au])C([O-])=O


In [15]:
print(patient["SMILES (drugA)"].isnull().sum())
print(patient["SMILES (drugB)"].isnull().sum())

0
0


In [16]:
# Fill missing values with an empty string
#df['smiles1'] = df['smiles1'].fillna('')

# Alternatively, drop rows with missing values
patient = patient.dropna(subset=['SMILES (drugA)', 'SMILES (drugB)'])

In [17]:
patient['molA'] = patient['SMILES (drugA)'].apply(Chem.MolFromSmiles)
patient['molB'] = patient['SMILES (drugB)'].apply(Chem.MolFromSmiles)
patient.head()

,patientID,gender,age,weight,prescribed drug (drugA),chem structure (drugA),newly prescribed drug (drugB),chem structure (drugB),DDI,clinical insights (recommendation),SMILES (drugA),SMILES (drugB),molA,molB
0,1940200,1,64,65,POTASSIUM CATION,K,LOTEPREDNOL ETABONATE,C24H31ClO7,moderate,NaN,[K+],[H][C@@]12CC[C@](OC(=O)OCC)(C(=O)OCCl)[C@@]1(C...,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...
1,1947086,0,55,62,ZINC SULFATE,O4SZn,POTASSIUM HYDROXIDE,HKO,no interaction found,NaN,[Zn++].[O-]S([O-])(=O)=O,[OH-].[K+],<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...
2,1947690,0,56,55,LINAGLIPTIN,C25H28N8O2,MEBENDAZOLE,C16H13N3O3,no interaction found,NaN,CC#CCN1C(=NC2=C1C(=O)N(CC1=NC3=C(C=CC=C3)C(C)=...,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=CC=C1,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...
4,1969141,1,40,85,PRILOCAINE,C13H20N2O,SODIUM AUROTHIOMALATE,C4H3AuNa2O4S,no interaction found,NaN,CCCNC(C)C(=O)NC1=CC=CC=C1C,[Na+].[Na+].[O-]C(=O)CC(S[Au])C([O-])=O,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...
5,1999592,1,68,66,AMPICILLIN,C16H19N3O4S,ORCIPRENALINE,C11H17NO3,no interaction found,NaN,[H][C@]12SC(C)(C)[C@@H](N1C(=O)[C@H]2NC(=O)[C@...,CC(C)NCC(O)C1=CC(O)=CC(O)=C1,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...


In [20]:
from rdkit.Chem import rdFingerprintGenerator
# Create a Morgan fingerprint generator
fpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=1024)

patient['fpA'] = patient['molA'].apply(fpgen.GetFingerprint)
patient['fpB'] = patient['molB'].apply(fpgen.GetFingerprint)
patient.head()

,patientID,gender,age,weight,prescribed drug (drugA),chem structure (drugA),newly prescribed drug (drugB),chem structure (drugB),DDI,clinical insights (recommendation),SMILES (drugA),SMILES (drugB),molA,molB,fpA,fpB
0,1940200,1,64,65,POTASSIUM CATION,K,LOTEPREDNOL ETABONATE,C24H31ClO7,moderate,NaN,[K+],[H][C@@]12CC[C@](OC(=O)OCC)(C(=O)OCCl)[C@@]1(C...,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1947086,0,55,62,ZINC SULFATE,O4SZn,POTASSIUM HYDROXIDE,HKO,no interaction found,NaN,[Zn++].[O-]S([O-])(=O)=O,[OH-].[K+],<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1947690,0,56,55,LINAGLIPTIN,C25H28N8O2,MEBENDAZOLE,C16H13N3O3,no interaction found,NaN,CC#CCN1C(=NC2=C1C(=O)N(CC1=NC3=C(C=CC=C3)C(C)=...,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=CC=C1,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
4,1969141,1,40,85,PRILOCAINE,C13H20N2O,SODIUM AUROTHIOMALATE,C4H3AuNa2O4S,no interaction found,NaN,CCCNC(C)C(=O)NC1=CC=CC=C1C,[Na+].[Na+].[O-]C(=O)CC(S[Au])C([O-])=O,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,1999592,1,68,66,AMPICILLIN,C16H19N3O4S,ORCIPRENALINE,C11H17NO3,no interaction found,NaN,[H][C@]12SC(C)(C)[C@@H](N1C(=O)[C@H]2NC(=O)[C@...,CC(C)NCC(O)C1=CC(O)=CC(O)=C1,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,<rdkit.Chem.rdchem.Mol object at 0x0000028EB53...,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [21]:
# Calculate Tanimoto coefficient
patient['tanimoto'] = patient.apply(lambda row: TanimotoSimilarity(row['fpA'], row['fpB']), axis=1)

In [23]:
# Print the results
print(patient[['SMILES (drugA)', 'SMILES (drugB)', 'tanimoto']])

                                        SMILES (drugA)  \
0                                                 [K+]   
1                             [Zn++].[O-]S([O-])(=O)=O   
2    CC#CCN1C(=NC2=C1C(=O)N(CC1=NC3=C(C=CC=C3)C(C)=...   
4                           CCCNC(C)C(=O)NC1=CC=CC=C1C   
5    [H][C@]12SC(C)(C)[C@@H](N1C(=O)[C@H]2NC(=O)[C@...   
..                                                 ...   
218       OC(=O)CNC(=O)C1=C(O)C=C(C=N1)C1=CC(Cl)=CC=C1   
219                  CNCCC=C1C2=CC=CC=C2CCC2=CC=CC=C12   
220  CC(C)OC(=O)CNC1=CC=CC(CN(CC2=CC=C(C=C2)N2C=CC=...   
221  [H][C@]12CCCC3=C1C(=CC=C3)C(=O)N(C2)[C@@H]1CN2...   
222  [H][C@@]12CC[C@](OC(=O)CC)(C(=O)CO)[C@@]1(C)CC...   

                                        SMILES (drugB)  tanimoto  
0    [H][C@@]12CC[C@](OC(=O)OCC)(C(=O)OCCl)[C@@]1(C...  0.000000  
1                                           [OH-].[K+]  0.000000  
2        COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=CC=C1  0.118280  
4              [Na+].[Na+].[O-]C(=O

In [24]:
patient.to_csv('patient_tanimoto.csv', index=False)